# Gallery Example: Hyper/Erl/k Queue

This example demonstrates a HyperExp/Erlang/k queueing system:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

This multiserver queue combines high arrival variability with regular service times.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hyperlk(k=2):
    """Create Hyper/Erl/k queueing model"""
    model = Network('Hyper/Erl/k')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    # Balanced HyperExp arrival with mean=1/1.8 and SCV=4
    source.set_arrival(oclass, HyperExp.fit_mean_and_scv_balanced(1/1.8, 4))
    # Erlang service with mean=1 and SCV=0.25 (order 4)
    queue.set_service(oclass, Erlang.fit_mean_and_scv(1, 0.25))
    queue.set_number_of_servers(k)
    
    # Block 3: topology
    model.link(Network.serial_routing(source, queue, sink))
    
    return model

# Create the model with k=2 servers
model = gallery_hyperlk(k=2)

## About Hyper/Erl/k Queues

This model combines:
- **High arrival variability** (SCV = 4): Bursty traffic
- **Low service variability** (SCV = 0.25): Regular service times
- **Multiple servers**: Parallel processing capacity

Parameters:
- Arrival mean = 1/1.8 ≈ 0.556 → arrival rate λ = 1.8
- Service mean = 1.0 → service rate μ = 1.0 per server
- With k=2 servers: total capacity = 2.0
- Utilization ρ = 1.8 / 2.0 = 0.9

The regular Erlang service helps counteract the high arrival variability.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)